# Avance y retroceso de los glaciares

Objetivos de este cuaderno:

- Comprender el concepto de altitud de línea de equilibrio (ELA)
- Comprender la relación entre el balance de masa glaciar y el ELA
- Ser capaz de explicar el avance y retroceso de los glaciares en respuesta a un cambio en el ELA

In [ ]:
from oggm_edu import GlacierBed, Glacier, MassBalance, GlacierCollection
from oggm_edu.funcs import plot_glacier_graphics

## Explorando el avance y retroceso de los glaciares con OGGM-Edu
Comenzamos definiendo un glaciar idealizado, luego dejamos que el glaciar crezca hasta que alcance su estado de equilibrio. Entonces podemos simular el avance y retroceso del glaciar. Representamos los diferentes pasos ilustrados en el [open glacier graphics from the OGGM-EDU website](https://edu.oggm.org/en/latest/glacier_basics.html) (hecho por Anne Maussion, [open glacier graphics from the OGGM-EDU website](https://edu.oggm.org/en/latest/glacier_basics.html)).

### Inicialización: Definamos nuestro glaciar

Primero, como siempre, definimos un perfil de lecho rocoso lineal:
A menudo, los glaciares son más anchos en el área de acumulación, por lo que construimos nuestro glaciar de manera que sea más ancho por encima del ELA.

In [ ]:
# Lets define the bed.
# Top and bottom altitude.
top = 5000
bottom = 0
# How far does the accumulation area extend?
accumulation_fraction = 1/3
# ELA altitude
ela_alt = (top - bottom) * (1 - accumulation_fraction)
# Accumulation area starts out as 1000 m. wide.
top_width = 1000
# 500 m. wide after ELA.
ela_width = 500

# Bed, note that we increase the map_dx here,
# we increase the grid point spacing.
bed = GlacierBed(altitudes=[top, ela_alt, bottom],
                 widths=[top_width, ela_width, ela_width],
                 map_dx=200)

In [ ]:
bed.plot()

Ahora necesitamos definir la distribución del balance de masa sobre el glaciar, después de lo cual el glaciar puede crecer hasta su estado de equilibrio inicial.

#### **Balance de masa y altitud de la línea de equilibrio**

El **balance de masa** es el resultado de varios procesos que agregan masa al glaciar (**acumulación**) o eliminan masa del glaciar (**ablación**). Puede encontrar más detalles en el [acumulación y ablación notebook](accumulation_and_ablation_es.ipynb). Como resumen, los siguientes gráficos de glaciares ilustran esta relación:
- El gráfico de la izquierda representa una acumulación de masa teórica sobre todo el glaciar representado por el volumen de hielo azul sobre el cuerpo gris del glaciar.
- En el gráfico central, una ablación de masa teórica está marcada como volumen de hielo rojo.
- El gráfico de la derecha muestra el balance de masa resultante con flechas azules y rojas en combinación con los cambios de volumen azul y rojo en la parte superior del cuerpo gris del glaciar. En la parte central del glaciar, donde la línea roja se encuentra directamente sobre el cuerpo gris del glaciar, la ablación y la acumulación se cancelaron entre sí.
<img src="https://raw.githubusercontent.com/OGGM/glacier-graphics/master/glacier_intro/png/glacier_03.png" width="33%" align="left">
<img src="https://raw.githubusercontent.com/OGGM/glacier-graphics/master/glacier_intro/png/glacier_04.png" width="33%" align="left">
<img src="https://raw.githubusercontent.com/OGGM/glacier-graphics/master/glacier_intro/png/glacier_05.png" width="33%" align="left">
*Fuente: [open glacier graphics](http://edu.oggm.org/en/latest/glacier_basics.html) en OGGM-Edu, realizado por Anne Maussion ([open glacier graphics](http://edu.oggm.org/en/latest/glacier_basics.html))*.

Las tasas de los procesos de acumulación y ablación, sumadas sobre el glaciar y a lo largo del tiempo, determinan el *balance de masa del glaciar*: $\dot{m}$, el cambio en la masa total de nieve y hielo,

$$\dot{m} = \text{acumulación} + \text{ablación}.$$

Dado que la acumulación y la ablación generalmente varían con la altura, el balance de masa del glaciar también es una función de la elevación,

$$\dot{m}(z) = \text{acumulación}(z) + \text{ablación}(z).$$

La masa se redistribuye continuamente en un glaciar: la masa acumulada en la parte superior del glaciar se transporta hacia abajo. La fuerza impulsora de este *flujo de hielo* es la gravedad. Por lo tanto, el balance de masa de una región en un glaciar depende no solo de los intercambios de masa inducidos por la acumulación y la ablación, sino también del transporte de hielo impulsado por la gravedad desde el área de acumulación al área de ablación. El *flujo de hielo* está indicado por la flecha gris en esta figura:
<img src="https://raw.githubusercontent.com/OGGM/glacier-graphics/master/glacier_intro/png/glacier_06.png" width="60%">

La altitud donde $\dot{m}(z) = 0$ se denomina *altitud de la línea de equilibrio*, abreviatura de ELA. Por lo tanto, el ELA es la altitud donde los procesos de acumulación y los procesos de ablación se equilibran entre sí, en teoría. Sin embargo, en realidad el ELA no existe exactamente y solo puede ser aproximado.
<img src="https://raw.githubusercontent.com/OGGM/glacier-graphics/master/glacier_intro/png/glacier_07.png" width="60%">

**Queremos reproducir estos procesos anteriores en un experimento usando OGGM-Edu**.
Para este propósito comenzamos con la definición de un balance de masa lineal que tiene la forma
$$ \ punto {m} (z) = (z - ELA) \ frac {d \ punto {m}} {dz}, $$
con el gradiente de balance de masas $\frac{d\dot{m}}{dz}$:

In [ ]:
# mass balance gradient with respect to elevation in mm w.e. m^-1 yr^-1
mb_grad = 7

Defina el ELA para que esté lo más cerca posible del ancho del ELA. Con esto podemos definir el modelo de balance de masa.

In [ ]:
# Defining the mass balance, we use the altitude defined above.
mass_balance = MassBalance(ela=ela_alt, gradient=mb_grad)

Ahora que tenemos todos los ingredientes para un glaciar, podemos inicializarlo.

In [ ]:
my_glacier = Glacier(bed=bed, mass_balance=mass_balance)
my_glacier

In [ ]:
my_glacier.plot()

### Glaciar en Equilibrio

Para que un glaciar esté en equilibrio, requerimos que el balance de masa específico (acumulación + ablación) tenga un promedio de cero durante un año en el glaciar. Un glaciar está en equilibrio si el glaciar no retrocede ni avanza de un año al siguiente si el clima se mantiene constante.

Avancemos el glaciar hasta que alcance el equilibrio

In [ ]:
my_glacier.progress_to_equilibrium()

Echa un vistazo al estado actual de los glaciares

In [ ]:
my_glacier.plot()

Aunque nuestro glaciar inicial está en equilibrio, no significa que el balance de masa sea cero en todas partes.
El balance de masa positivo neto en la parte superior (más acumulación, menos ablación) equilibra el balance de masa negativo neto en la lengua (menos acumulación, más ablación).
Como se explica en el [acumulación y ablación notebook](accumulation_and_ablation_es.ipynb), la acumulación neta en la parte superior da como resultado un flujo de hielo a través del glaciar que compensa el derretimiento en las áreas inferiores.
**En equilibrio, el balance de masa neto de un glaciar es cero, pero el hielo sigue moviéndose de arriba hacia abajo.**
Ahora hemos preparado el escenario para echar un vistazo al avance y retroceso del glaciar.

## Glaciar que avanza

Para inducir el avance de un glaciar utilizaremos el mismo glaciar que antes, pero bajando la temperatura.
Esto mueve el ELA por el glaciar.
Para lograr esto configuramos un escenario de cambio climático con el método `.add_temperature_bias`.
Nuestro escenario implica una disminución de temperatura de -1.5$^\circ$C durante 50 años.

In [ ]:
my_glacier.add_temperature_bias(bias=-1.5, duration=50)

Luego avanzamos el glaciar hasta que alcanza el equilibrio.

In [ ]:
my_glacier.progress_to_equilibrium()

In [ ]:
my_glacier.mass_balance

Como podemos ver en la representación anterior, el ELA actual se encuentra alrededor de los 3108 m s.n.m. mientras que el ELA original estaba a 3333 m s.n.m.
Echemos un vistazo al nuevo estado de equilibrio después de que redujimos la temperatura.

In [ ]:
my_glacier.plot_state_history(eq_states=True)

La disminución de la ELA da como resultado un área de acumulación aumentada y, por lo tanto, una mayor acumulación. Más masa en la parte superior del glaciar aumentará el flujo de hielo $\vec{q}$ hacia abajo del glaciar, lo que provocará un avance del extremo del glaciar y, por lo tanto, expandirá el área de ablación.
Podemos ver esto observando los contornos en el diagrama de arriba.
El contorno azul, más antiguo, se extiende más abajo en el glaciar en comparación con el contorno naranja, más joven.
Esto también se ilustra en la siguiente gráfica.

In [ ]:
plot_glacier_graphics('09');

## Glaciar en retirada

Del mismo modo, podemos provocar el retroceso del glaciar.
Estableceremos nuestro glaciar inicial nuevamente, pero esta vez aumentaremos la temperatura y así moveremos el ELA hacia arriba del glaciar.
Para ello planteamos otro escenario de cambio climático, pero esta vez con un cambio de temperatura positivo.

In [ ]:
mass_balance = MassBalance(ela=ela_alt, gradient=mb_grad)
my_glacier = Glacier(bed=bed, mass_balance=mass_balance)
my_glacier.progress_to_equilibrium()

In [ ]:
# Climate change scenario
# 3 degrees warmer in 100 years.
my_glacier.add_temperature_bias(bias=3., duration=100)

Y progresemos hasta llegar a un nuevo equilibrio

In [ ]:
my_glacier.progress_to_equilibrium()

In [ ]:
my_glacier.mass_balance

Aquí acabamos elevando la ELA desde 3333 m. asl a 3783 m. asl

Trazar el nuevo estado de equilibrio

In [ ]:
my_glacier.plot_state_history(eq_states=True)

Comparando los contornos de los dos estados de equilibrio, podemos ver que el glaciar ha retrocedido: el contorno naranja en corto es más antiguo en comparación con el contorno azul.
Similar a lo que vemos en la siguiente figura.

In [ ]:
plot_glacier_graphics('10');

Podemos explorar más a fondo la respuesta del glaciar durante la retirada observando la historia de nuestro glaciar.

In [ ]:
my_glacier.plot_history()

El aumento de la temperatura hará crecer el área de ablación que corresponde al aumento de la ELA.
Esto da como resultado una mayor pérdida de masa y un menor flujo de hielo para el glaciar.
Dado que la acumulación ya no puede seguir el ritmo de la mayor pérdida de masa, el glaciar comenzará a retroceder.
Finalmente, el glaciar ha retrocedido lo suficiente como para que la ablación anual se equilibre una vez más con la acumulación anual. $\rightarrow$ En este punto, el glaciar ha alcanzado un nuevo estado de equilibrio.


## Llévate puntos a casa

- La altitud de la línea de equilibrio (ELA) es la altitud en un glaciar donde la acumulación y la ablación están en equilibrio, $\dot{m}(z) = 0$ en $z=$ ELA
- Una disminución en el ELA conduce a:
1. Mayor acumulación
2. Una disminución inicial del área de ablación
3. Una ganancia de masa neta que resulta en un aumento del flujo de hielo por el glaciar.
4. Avance del glaciar
- Un aumento en el ELA conduce a:
1. Un área de acumulación reducida
2. Un área de ablación aumentada
3. Una pérdida de masa neta que resulta en una disminución del flujo de hielo por el glaciar.
4. Retiro del glaciar

## ¿Que sigue?

xx_markdown_enlace_xx